# Reading live camera data

In [3]:
# Import the required modules
import cv2
import numpy as np
from IPython.display import clear_output, display, Image
import PIL.Image
from io import BytesIO
import ipywidgets as widgets

def img2ByteArr(frame, ext='jpeg'):
    byteObj = BytesIO()
    PIL.Image.fromarray(frame).save(byteObj, ext)
    return byteObj.getvalue()

cam = cv2.VideoCapture(0) #Resolutions from camera- 1920x1080, 1280x720, 640x360
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 360)
_, frame = cam.read()

#Create Ipython image widget
w=widgets.Image(value=img2ByteArr(frame));
display(w)


#print(widgets.height, widgets.width)

for frame_number in range(100):
    #Capture frame
    _, frame = cam.read()
    #print(frame.shape)
    
    #Perform operation
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    #Display frame
    w.value=img2ByteArr(frame)

cam.release()
clear_output()
#print(frame.shape)
